# 1. User Authentication

### Registration
- register_user(): This function handles the registration process. It takes the username and password as input, ensures the username is unique using the is_username_unique() function, hashes the password using the hash_password() function, and then stores the username and hashed password in a file (USER_CREDENTIALS_FILE).    
- is_username_unique(): This function checks if the entered username already exists in the USER_CREDENTIALS_FILE.    
- hash_password(): This function takes the password as input and returns its hashed version using the SHA-256 algorithm.

In [1]:
import hashlib
import os

USER_CREDENTIALS_FILE = "user_credentials.txt"

def is_username_unique(username):
    """Checks if a username already exists in the user credentials file."""
    if not os.path.exists(USER_CREDENTIALS_FILE):
        return True  # File doesn't exist, so username is unique

    with open(USER_CREDENTIALS_FILE, "r") as file:
        for line in file:
            existing_username, _ = line.strip().split(":")
            if existing_username == username:
                return False  # Username exists
    return True  # Username is unique

def hash_password(password):
    """Hashes the given password using SHA-256."""
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    return hashed_password

def register_user():
    """Prompts the user for a username and password, 
    and stores the credentials securely."""

    while True:
        username = input("Enter a username: ")
        if is_username_unique(username):  # Check if username is unique
            break
        else:
            print("Username already exists. Choose another.")

    password = input("Enter a password: ")
    hashed_password = hash_password(password)  # Hash the password

    with open(USER_CREDENTIALS_FILE, "a") as file:
        file.write(f"{username}:{hashed_password}\n")
    print("Registration successful.")

register_user() #call register user to run the code.

Registration successful.


### Login
- login_user(): This function handles the login process. It prompts the user for their username and password, hashes the entered password, and then calls the verify_credentials() function to validate the credentials. If the credentials are valid, it grants access; otherwise, it displays an error message.    
- verify_credentials(): This function reads the stored usernames and hashed passwords from the USER_CREDENTIALS_FILE and compares them with the entered credentials to validate the user.

In [3]:
def login_user():
    """Prompts the user for username and password, 
    and validates the credentials."""

    username = input("Enter your username: ")
    password = input("Enter your password: ")
    hashed_password = hash_password(password)

    if verify_credentials(username, hashed_password):
        print("Login successful.")
        return username  # Return username for further use
    else:
        print("Invalid username or password.")
        return None  # Login failed

def verify_credentials(username, hashed_password):
    """Verifies the provided credentials against stored data."""

    if not os.path.exists(USER_CREDENTIALS_FILE):
        return False  # No users registered

    with open(USER_CREDENTIALS_FILE, "r") as file:
        for line in file:
            stored_username, stored_hashed_password = line.strip().split(":")
            if stored_username == username and stored_hashed_password == hashed_password:
                return True  # Credentials match
    return False  # No match
login_user() #call login user to run the code.

Login successful.


'sergelafarge'

# 2. Add a Task
- add_task(): This function prompts the user for a task description, generates a unique task ID using generate_unique_task_id(), sets the initial status of the task to "Pending," and then saves the task to the user's task file using the save_task() function.    
- generate_unique_task_id(): This function generates a unique ID for each task for a user.    
- save_task(): This function saves the task details (task ID, description, and status) to the user's task file.    
- get_task_file_path(): This function constructs the file path for a user's task file.

In [11]:
import os

TASKS_DIRECTORY = r"C:\Users\serge\OneDrive\Desktop\AIML Specialization\Mandatory Courses\PC AIML - Programming Refresher\Projects\tasks"  # Absolute path

def add_task(username):
    """Prompts the user for a task description,
    assigns a unique ID, and adds the task."""

    task_description = input("Enter the task description: ")
    task_id = generate_unique_task_id(username)  # Generate a unique Task ID
    status = "Pending"

    save_task(username, task_id, task_description, status)
    print("Task added successfully.")

def generate_unique_task_id(username):
    """Generates a unique task ID for the user."""

    task_file = get_task_file_path(username)
    if not os.path.exists(task_file):
        return 1  # Start ID from 1 if no tasks exist yet
    try:
        with open(task_file, "r") as file:
            lines = file.readlines()
            if not lines:
                return 1 # File exisits but is empty
            
            last_line = lines[-1].strip()
            parts = last_line.split(":")
            
            if len(parts) > 0 and parts[0].isdigit(): # check that parts[0] exists and is a digit
                last_task_id = int(parts[0])
                return last_task_id + 1  # Increment the last task ID
            else:
                # If the last line doesn't have a valid task ID, start from 1
                return 1
            
    except FileNotFoundError:
        return 1
    
def save_task(username, task_id, task_description, status):
    """Saves the task to the user's task file."""

    task_file = get_task_file_path(username)
    with open(task_file, "a") as file:
        file.write(f"{task_id}:{task_description}:{status}\n")

def get_task_file_path(username):
    """Constructs the file path for a user's task file."""
    return os.path.join(TASKS_DIRECTORY, f"{username}_tasks.txt")

# Example usage:
username = "sergelafarge"  # Replace with the desired username
add_task(username) # Call add_task to run the code.

Task added successfully.


# 3. View Tasks
- view_tasks(): This function retrieves all tasks for the logged-in user from their task file and displays them in a formatted way, showing the task ID, description, and status.

In [14]:
def view_tasks(username):
    """Retrieves and displays all tasks for the logged-in user."""
    
    task_file = get_task_file_path(username)
    if not os.path.exists(task_file):
        print("No tasks found.")
        return
    
    with open(task_file, "r") as file:
        tasks = file.readlines()
        if not tasks:
            print("No tasks found.")
            return
    
        print("Tasks:")
        for task in tasks:
            task_id, task_description, status = task.strip().split(":")
            print(f"  ID: {task_id}, Description: {task_description}, Status: {status}")
view_tasks(username) # Call view_tasks to run the code.

Tasks:
  ID: sergelafarge_tasks1, Description: sergelafarge, Status: Pending


# 4. Mark a Task as Completed
- mark_task_as_completed(): This function prompts the user to enter the ID of the task they want to mark as completed and then calls the update_task_status() function to update the task's status.    
- update_task_status(): This function reads the user's task file, finds the task with the given ID, and changes its status to "Completed".

In [17]:
def mark_task_as_completed(username):
    """Allows the user to select a task by ID 
    and update its status to Completed."""
    
    task_id = input("Enter the ID of the task to mark as completed: ")
    update_task_status(username, task_id, "Completed")
    
def update_task_status(username, task_id, new_status):
    """Updates the status of the specified task."""
    
    task_file = get_task_file_path(username)
    if not os.path.exists(task_file):
        print("No tasks found.")
        return
    
    updated_tasks = []
    task_found = False
    with open(task_file, "r") as file:
        for line in file:
            stored_task_id, task_description, status = line.strip().split(":")
            if stored_task_id == task_id:
                updated_tasks.append(f"{stored_task_id}:{task_description}:{new_status}\n")
                task_found = True
            else:
                updated_tasks.append(line)
    
    if task_found:
        with open(task_file, "w") as file:
            file.writelines(updated_tasks)
        print("Task status updated.")
    else:
        print("Task not found.")
mark_task_as_completed(username) # Call mark_task_as_completed to run the code.

Task status updated.


# 5. Delete a Task
- delete_task(): This function prompts the user for the ID of the task they want to delete and calls the remove_task() function to remove it.    
- remove_task(): This function reads the user's task file and removes the task with the specified ID. 
- intentionally did not delete the task

In [18]:
def delete_task(username):
    """Allows the user to select a task by ID and delete it."""
    
    task_id = input("Enter the ID of the task to delete: ")
    remove_task(username, task_id)
    
def remove_task(username, task_id):
    """Removes the specified task from the user's task list."""
    
    task_file = get_task_file_path(username)
    if not os.path.exists(task_file):
        print("No tasks found.")
        return
    
    updated_tasks = []
    task_found = False
    with open(task_file, "r") as file:
        for line in file:
            stored_task_id, task_description, status = line.strip().split(":")
            if stored_task_id != task_id:  # Keep tasks that are NOT the one to delete
                updated_tasks.append(line)
            else:
                task_found = True
    
    if task_found:
        with open(task_file, "w") as file:
            file.writelines(updated_tasks)
        print("Task deleted.")
    else:
        print("Task not found.")
delete_task(username) # Call delete_task to run the code.

Task not found.


# 6. Create an Interactive Menu
- main_menu(): This function displays the main menu with options to add, view, mark as complete, and delete tasks, and to logout. It takes the user's choice as input and calls the corresponding function. The menu loop continues until the user chooses to logout.    
- The if __name__ == "__main__": block is the main program flow. It first creates the TASKS_DIRECTORY if it doesn't exist. Then, it enters a loop to present the user with the initial options: register, login, or exit. After successful login, it calls the main_menu() function to allow the user to manage their tasks.

In [20]:
def main_menu(username):
    """Displays the main menu and handles user input."""
    
    while True:
        print("\nTask Manager Menu:")
        print("1. Add a Task")
        print("2. View Tasks")
        print("3. Mark a Task as Completed")
        print("4. Delete a Task")
        print("5. Logout")
    
        choice = input("Enter your choice: ")
    
        if choice == "1":
            add_task(username)
        elif choice == "2":
            view_tasks(username)
        elif choice == "3":
            mark_task_as_completed(username)
        elif choice == "4":
            delete_task(username)
        elif choice == "5":
            break  # Exit the loop (logout)
        else:
            print("Invalid choice. Please try again.")

# Main program flow
if __name__ == "__main__":
    # Create the tasks directory if it doesn't exist
    if not os.path.exists(TASKS_DIRECTORY):
        os.makedirs(TASKS_DIRECTORY)
    
    while True:
        print("\nWelcome to the Task Manager!")
        print("1. Register")
        print("2. Login")
        print("3. Exit")
    
        choice = input("Enter your choice: ")
    
        if choice == "1":
            register_user()
        elif choice == "2":
            username = login_user()
            if username:  # If login successful
                main_menu(username)
        elif choice == "3":
            break  # Exit the program
        else:
            print("Invalid choice. Please try again.")
    
    print("Thank you for using the Task Manager!")
# End of the script



Welcome to the Task Manager!
1. Register
2. Login
3. Exit
Thank you for using the Task Manager!
